あらゆる情報をまずは表にして，それから可視化する

- 

# Association Rule によるルール作成

In [1]:
import json 
import pandas as pd

In [2]:
# data_dir_path = "/Users/keisukeonoue/ws/lukasiewicz_2/experiments"
result_file_path_list = [
    "./experiments/version_153/result.json",
    "./experiments/version_154/result.json",
    "./experiments/version_155/result.json",
    "./experiments/version_156/result.json",
]

In [3]:
result_file_path = result_file_path_list[0]

with open(result_file_path, 'r') as f:
    json_data = json.load(f)

json_data

{'path': './experiments/version_153',
 'source_paths': ['data/pima_indian_diabetes/diabetes_cleaned_normalized.csv',
  'data/pima_indian_diabetes/diabetes_discretized.csv'],
 'experiment_name': 'pima_indian_diabetes_cv_15',
 'seed': 42,
 'n_splits': 5,
 'n_unsupervised': 15,
 'c1': 10,
 'c2': 10,
 'rule_thr': 0.4,
 'result': {'fold_0': {'RuleFit Classifier (disc)': {'accuracy': 0.7761194029850746,
    'precision': 0.7333333333333333,
    'recall': 0.5,
    'f1': 0.5945945945945945,
    'auc': 0.8353535353535353,
    'violation_detail': {'0': [26, 1, [83]]},
    'n_violation': 1,
    'n_rule': 1,
    'violation_rate': 1.0,
    'n_violation (instance)': 1,
    'n_evaluation (instance)': 26,
    'violation_rate (instance)': 0.038461538461538464},
   'tree generator (disc)': {'accuracy': 0.7761194029850746,
    'precision': 0.675,
    'recall': 0.6136363636363636,
    'f1': 0.6428571428571429,
    'auc': 0.8170454545454545,
    'violation_detail': {'0': [26, 5, [0, 6, 83, 92, 124]]},
    '

In [21]:
for i in json_data:
    print(i)

print()

for i in json_data['result']:
    print(i)

print()

for i in json_data['result']['fold_0']:
    print(i)

path
source_paths
experiment_name
seed
n_splits
n_unsupervised
c1
c2
rule_thr
result

fold_0
fold_1
fold_2
fold_3
fold_4

RuleFit Classifier (disc)
tree generator (disc)
RuleFit Classifier (conti)
tree generator (conti)
linear svm (L)
non-linear svm (L)
logistic regression (L)
linear svm
non-linear svm
logistic regression
rule_based_prediction


In [22]:
for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']:
    print(i)

print()

for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']['violation_detail']:
    print(i)

print()

for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']['violation_detail']['0']:
    print(i)

accuracy
precision
recall
f1
auc
violation_detail
n_violation
n_rule
violation_rate
n_violation (instance)
n_evaluation (instance)
violation_rate (instance)

0

26
1
[83]


## 一時保存した実験結果の読み込み

In [4]:
result_dfs = []
infos = []

for path in result_file_path_list:
    with open(path, 'r') as f:
        json_data = json.load(f)
    
    infos.append(json_data['rule_thr'])
    
    tmp = []

    for fold, _ in json_data['result'].items():
        result_df = pd.DataFrame(json_data['result'][fold]).T.drop(['violation_detail'], axis=1)
        tmp.append(result_df)

    result_dfs.append(tmp)

print(len(result_dfs))
print(len(result_dfs[0]))
print()

model_name_list = [
    'RuleFit Classifier (disc)', 
    'tree generator (disc)', 
    'RuleFit Classifier (conti)', 
    'tree generator (conti)', 
    'linear svm (L)', 
    'non-linear svm (L)', 
    'logistic regression (L)', 
    'linear svm', 
    'non-linear svm', 
    'logistic regression', 
    'rule_based_prediction'
]

combined_dfs = [pd.concat(dfs, ignore_index=False) for dfs in result_dfs]
dfs_mean = [combined_df.groupby(combined_df.index).mean().reindex(index=model_name_list) for combined_df in combined_dfs]
dfs_std = [combined_df.groupby(combined_df.index).std().reindex(index=model_name_list) for combined_df in combined_dfs]

print(len(dfs_mean))

4
5

4


## 実験結果の保存（クロスバリデーション）

In [24]:
import os

for version, (dfs, info) in enumerate(zip(result_dfs, infos)):
    
    for nth_fold, df in enumerate(dfs):

        df['nth_fold'] = nth_fold
        df['n_splits'] = 5
        df['C1'] = 10
        df['C2'] = 10
        df['n_unsupervised'] = 15
        df['rule_thr'] = info

        # フォルダーを作成する
        folder_path = f'./experiment_15/fold_{nth_fold}'
        os.makedirs(folder_path, exist_ok=True)

        file_path = f'{folder_path}/ver_{version}.csv'
        print(file_path)

        df.to_csv(file_path)


./experiment_15/fold_0/ver_0.csv
./experiment_15/fold_1/ver_0.csv
./experiment_15/fold_2/ver_0.csv
./experiment_15/fold_3/ver_0.csv
./experiment_15/fold_4/ver_0.csv
./experiment_15/fold_0/ver_1.csv
./experiment_15/fold_1/ver_1.csv
./experiment_15/fold_2/ver_1.csv
./experiment_15/fold_3/ver_1.csv
./experiment_15/fold_4/ver_1.csv
./experiment_15/fold_0/ver_2.csv
./experiment_15/fold_1/ver_2.csv
./experiment_15/fold_2/ver_2.csv
./experiment_15/fold_3/ver_2.csv
./experiment_15/fold_4/ver_2.csv
./experiment_15/fold_0/ver_3.csv
./experiment_15/fold_1/ver_3.csv
./experiment_15/fold_2/ver_3.csv
./experiment_15/fold_3/ver_3.csv
./experiment_15/fold_4/ver_3.csv


## 実験結果の保存（クロスバリデーション，平均と標準偏差）

In [25]:
for version, (info, df) in enumerate(zip(infos, dfs_mean)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = f'./experiment_15/ver_{version}_mean.csv'
    print(file_path)

    df.to_csv(file_path)

for version, (info, df) in enumerate(zip(infos, dfs_std)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = f'./experiment_15/ver_{version}_std.csv'
    print(file_path)

    df.to_csv(file_path)



./experiment_15/ver_0_mean.csv
./experiment_15/ver_1_mean.csv
./experiment_15/ver_2_mean.csv
./experiment_15/ver_3_mean.csv
./experiment_15/ver_0_std.csv
./experiment_15/ver_1_std.csv
./experiment_15/ver_2_std.csv
./experiment_15/ver_3_std.csv


## ルールの整形と保存（csv）

In [24]:
import ast

for nth_fold in range(5):
    for version in range(len(infos)):
        dir_path = f"./experiments/version_15{version + 3}"
        file_path_1 = f"{dir_path}/rules/rules_{nth_fold}.csv"
        file_path_2 = f"{dir_path}/rules/rules_{nth_fold}.txt"
        file_path_3 = f"{dir_path}/result.json"

        with open(file_path_3, 'r') as f:
            result = json.load(f)
        rule_result = result['result'][f'fold_{nth_fold}']['rule_based_prediction']['violation_detail']
        df_rule_result = pd.DataFrame(rule_result).T

        df = pd.read_csv(file_path_1, index_col=0)

        KB = []
        with open(file_path_2, 'r') as f:
            for line in f:
                KB.append(line.rstrip())        

        rows_tmp = []
        for i in range(len(df)):
            row_df = df.iloc[i]
            condition = ast.literal_eval(f"[{row_df['antecedents']}]")
            conclusions = ast.literal_eval(f"[{row_df['consequents']}]")

            for conclusion in conclusions:
                rule = condition + ['→'] + [conclusion] if row_df['lift'] >= 1 else condition + ['→', '¬'] + [conclusion]
                rule = " ".join(rule)
                row_df['rule'] = rule
                rows_tmp.append(row_df[['rule', 'support', 'confidence', 'lift']])

        df = pd.DataFrame(rows_tmp)
        display(df.head())


        # df['rule'] = KB
        # df = df[['rule', 'support', 'confidence', 'lift']]
        

        # df.drop(['type', 'support', 'importance'], axis=1, inplace=True)
        # df['rule'] = KB[:df.shape[0]]

        # n_predicate_list = [] 
        # implication_bool_list = []
        # for h, rule in enumerate(df['rule']):
        #     n_predicate_list.append(sum([1 for element in rule.split() if element not in ['⊗', '¬', '→']]))

        #     # consequent カラムのために後で追加
        #     if rule[rule.split().index('→') + 1] == '¬':

        #         implication_bool_list.append('¬')
        #     else:
        #         implication_bool_list.append(None)

        # df['n_predicate'] = n_predicate_list



        # df['n_violation (instance)'] = df_rule_result[1].to_list()
        # df['n_evaluation (instance)'] = df_rule_result[0].to_list()
        # df['violation'] = df['n_violation (instance)'].apply(lambda x: 0 if x == 0 else 1)
        # df['consequent'] = implication_bool_list

        # display(df.head(3))

        # ########################################################################
        # ########################################################################
        # ########################################################################
        # ########################################################################
        file_path = f'./experiment_15/fold_{nth_fold}/rules_{version}.csv'
        df.to_csv(file_path)


/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule']

,rule,support,confidence,lift
2,Glucose_High → Outcome,0.143657,0.719626,2.216779
68,SkinThickness_Low → BMI_Low,0.220149,0.506438,1.606217
69,BMI_Low → SkinThickness_Low,0.220149,0.698225,1.606217
141,BloodPressure_Low → Pregnancies_Low,0.130597,0.804598,1.404770
141,BloodPressure_Low → Age_Low,0.130597,0.804598,1.404770


/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule']

,rule,support,confidence,lift
2,Glucose_High → Outcome,0.143657,0.719626,2.216779
68,SkinThickness_Low → BMI_Low,0.220149,0.506438,1.606217
69,BMI_Low → SkinThickness_Low,0.220149,0.698225,1.606217
162,SkinThickness_Low Pregnancies_Low → BMI_Low,0.154851,0.538961,1.709368
164,Pregnancies_Low BMI_Low → SkinThickness_Low,0.154851,0.715517,1.645997


/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule']

...

,rule,support,confidence,lift
1,Glucose_High → Outcome,0.138060,0.711538,2.230319
157,SkinThickness_Low Pregnancies_Low → BMI_Low,0.166045,0.593333,1.827739
159,Pregnancies_Low BMI_Low → SkinThickness_Low,0.166045,0.729508,1.649858
160,BMI_Low → SkinThickness_Low,0.166045,0.511494,1.827739
160,BMI_Low → Pregnancies_Low,0.166045,0.511494,1.827739


/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule']

,rule,support,confidence,lift
1,Glucose_High → Outcome,0.138060,0.711538,2.230319
157,SkinThickness_Low Pregnancies_Low → BMI_Low,0.166045,0.593333,1.827739
160,BMI_Low → SkinThickness_Low,0.166045,0.511494,1.827739
160,BMI_Low → Pregnancies_Low,0.166045,0.511494,1.827739
294,SkinThickness_Low Age_Low → BMI_Low,0.173507,0.600000,1.848276


/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule'] = rule
/var/folders/s5/29r5x3qd5h30ntfm58ck0mx00000gn/T/ipykernel_1719/855681334.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_df['rule']

,rule,support,confidence,lift
1,Glucose_High → Outcome,0.138060,0.711538,2.230319
463,SkinThickness_Low Pregnancies_Low → Age_Low,0.145522,0.520000,2.144000
463,SkinThickness_Low Pregnancies_Low → BMI_Low,0.145522,0.520000,2.144000
464,SkinThickness_Low Age_Low → Pregnancies_Low,0.145522,0.503226,2.210894
464,SkinThickness_Low Age_Low → BMI_Low,0.145522,0.503226,2.210894


In [ ]:
# # モデルごとに予測が違

# for version in range(len(infos)):
#     file_path = f"./experiments/version_10{version + 1}/result.json"

#     with open(file_path, 'r') as f:
#         result = json.load(f)
    
#     result['result'][f'fold_{nth_fold}']

In [6]:
tmp = "('a', 'b')"
tmp

"('a', 'b')"

('(', "'", 'a', "'", ',', ' ', "'", 'b', "'", ')')

In [13]:
tmp = pd.DataFrame()
tmp2 = ['a', 'b', 'c']

for i in range(3):
    tmp.iloc[i] = tmp2

IndexError: iloc cannot enlarge its target object

In [10]:
tmp

,0,1,2
0,a,a,a
1,b,b,b
2,c,c,c


In [29]:
import pandas as pd

# サンプルのDataFrameを作成
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35]}
df = pd.DataFrame(data)

# 行を下にコピーする
new_row = df.iloc[0].copy()  # コピー元の行をコピー
df = df.append(new_row, ignore_index=True)

print(df)


AttributeError: 'DataFrame' object has no attribute 'append'

In [30]:
df

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35


In [46]:
tmp = []

for i in range(df.shape[0]):
    tmp.append(df.iloc[i])

In [49]:
pd.DataFrame(tmp)

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35


In [14]:
import ast

# リストを含む文字列
string_with_list = "'SkinThickness_Low', 'Pregnancies_Low'"

# 文字列をリストに変換
formatted_string = f"[{string_with_list}]"
result_list = ast.literal_eval(formatted_string)

print(result_list)


ast.literal_eval(f"[{}]")

['SkinThickness_Low', 'Pregnancies_Low']
